In [2]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt
import csv

### Tarea 1
Los ejemplos ilustrativos anteriores permiten saber el número de monedas presentes en la imagen. ¿Cómo saber la cantidad de dinero presente en ella? Sugerimos identificar de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras obtener esa información y las dimensiones en milímetros de las distintas monedas, realiza una propuesta para estimar la cantidad de dinero en la imagen. Muestra la cuenta de monedas y dinero sobre la imagen. No hay restricciones sobre utilizar medidas geométricas o de color.

Una vez resuelto el reto con la imagen ideal proporcionada, captura una o varias imágenes con monedas. Aplica el mismo esquema, tras identificar la moneda del valor determinado, calcula el dinero presente en la imagen. ¿Funciona correctamente? ¿Se observan problemas?

Nota: Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc.

Extras: Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.

In [ ]:
# Leer la imagen de las monedas
img = cv2.imread('./content/monedas_2.png') 

# Convierte la imagen a todos de gris, mostrando el resultado
img_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Umbralizado binario invertido
umbral = 200
th1,img_th1 = cv2.threshold(img_gris,umbral,255,cv2.THRESH_BINARY_INV)

# Buscamos los contornos
contours, hierarchy = cv2.findContours(img_th1, 
    cv2.RETR_EXTERNAL , 
    cv2.CHAIN_APPROX_SIMPLE)

# Guardamos las coordenadas del centro y el radio de cada círculo 
circles = []

for cnt in contours:
    epsilon = 0.1 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    (x,y), radius = cv2.minEnclosingCircle(approx)
    center = (int(x),int(y))
    radius = int(radius)
    if radius > 5:
        circles.append([center, radius])
    cv2.circle(img, center, radius, (255, 0, 0), 3)

valores = {
    2.00: 25.75,
    1.00: 23.25,
    0.50: 24.25,
    0.20: 22.25,
    0.10: 19.75,
    0.05: 21.25,
    0.02: 18.75,
    0.01: 16.25    
}

# Cogemos la moneda de mayor tamaño como referencia
mm = 25.75
# Pasamos a píxeles
px = max(circles, key=lambda x: x[1])[1]*2
escala = mm/px


# Función para detectar clics en la imagen
def click(event, x, y, flags, param):
    total = 0.00
    valor = 0.00
    # Si presionoo el botón izquierdo del ratón
    if event == cv2.EVENT_LBUTTONDOWN:
        # Recorro los contornos para ver si he hecho clic en alguno
        for (center, r) in circles:
            # x, y
            cx, cy = center
            # Si hago click dentro del radio
            if ((x - cx)**2 + (y - cy)**2) <= r**2:
                # Determino el valor de la moneda y su tamaño
                diam_mm = (r * 2) * escala
                valor = min(valores, key=lambda v: abs(valores[v] - diam_mm))
                print(f"Moneda: {valor:.2f}€  ({diam_mm:.2f} mm)")
                break
                

cv2.namedWindow("Testing", cv2.WINDOW_NORMAL)
ratio = (img.shape[1]) / 900
new_h = int(img.shape[0]//ratio)
new_w = int(img.shape[1]//ratio)
cv2.resizeWindow("Testing", new_h, new_w)
cv2.setMouseCallback("Testing", click)
cv2.imshow("Testing", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Moneda: 0.01€  (4.40 mm)


Tarea 2.

NOTA: Podemos usar técnicas de segmentación más potentes para distinguir los objetos del fondo. 
RECORDAR: No se espera una clasificación ni separación del 100%, si se consigue, pues genial, pero no es requisito.